# Research Computing Boot Camp
# Boston University 

### Website: [rcs.bu.edu](http://www.bu.edu/tech/support/research/) <br>
Tutorial materials: [https://github.com/bu-rcs/bu-rcs.github.io/tree/main/Bootcamp](https://github.com/bu-rcs/bu-rcs.github.io/tree/main/Bootcamp)

# Python Part 4:  Data Analysis

In this final tutorial we'll look at some tools used to do analyze data using Python, Pandas, and some additional libraries.

Not demo'd here, but as a reference this library is very popular for machine learning style data analysis in Python: scikit-learn (https://scikit-learn.org/stable/) 

From their website:
* Simple and efficient tools for predictive data analysis
* Accessible to everybody, and reusable in various contexts
* Built on NumPy, SciPy, and matplotlib

# Download data

Repeating the steps from part 3, let's get the complete data set...

In [ ]:
# Load our libraries.  It's good practice to put all library loading at the top of the notebook.
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 100 # for larger plots 

In [ ]:
# Let's import the US-wide health data.
df = pd.read_csv('https://raw.githubusercontent.com/bu-rcs/bu-rcs.github.io/main/Bootcamp/Data/USA_HealthData.csv')
# Clean out the NaN County rows, as before
df.drop(df[df['County'].isnull()].index, inplace=True) 
# And delete the FIPS column for plotting convenience
del df['FIPS']
# Bring in the census regions
reg_df = pd.read_csv('https://raw.githubusercontent.com/bu-rcs/bu-rcs.github.io/main/Bootcamp/Data/us_states_census_regions.csv')
df = pd.merge(left=df, right=reg_df, left_on='State', right_on='State')

# And the US-wide demographic data
demo_df = pd.read_csv('https://raw.githubusercontent.com/bu-rcs/bu-rcs.github.io/main/Bootcamp/Data/USA_DemographicsData.csv')
demo_df.drop(demo_df[demo_df['County'].isnull()].index, inplace=True) 
del demo_df['FIPS']

# Merge in the demographic data by county and state
df = pd.merge(left=df, right=demo_df, left_on=['County','State'], right_on=['County','State'])
# Let's rename a few columns for convenience
rename={"Life Expectancy":'life_exp', "% Frequent Physical Distress":'ph_distress',
        "% Limited Access to Healthy Foods":'limit_healthy_food',"# Black":'n_black',
       "# Rural":'n_rural',"Median Household Income":'house_income',
       'Population':'pop'}
df = df.rename(columns=rename)

# Preview
df.head()

## Zipf's Law

https://en.wikipedia.org/wiki/Zipf's_law

First, let's do some curve fitting (aka regression) to the data. Zipf's Law is a empirical law first observed in language: 
>...given a large sample of words used, the frequency of any word is inversely proportional to its rank in the frequency table. So word number n has a frequency proportional to 1/n.

As has been observed many times, this is true for things like the [population of cities](https://arxiv.org/ftp/arxiv/papers/1402/1402.2965.pdf) and the frequency of the size of the cities .  Let's see if we can observe Zipf's law this when looking at US county populations.

In [ ]:
# First we'll need the population of each county.  No need to work with the whole dataframe, let's just use 
# a Series. We want to then sort and modify this Series in-place (i.e. without making new Series or Dataframes)
# so force a copy to be made.

# Look at populations >10,000
pop = df[df['pop'] > 10000][['pop']].copy()

# Next, sort in descending order.
pop.sort_values(ascending=False, inplace=True,by='pop')
pop


In [ ]:
# We'll need an index starting at 1 to the size of the Series
pop['ind'] = np.arange(1,pop.size+1)

In [ ]:
# Now plot it!
ax=pop.plot(x='ind',y='pop')
ax.set_xlabel('Rank')
ax.set_ylabel('Population');

In [ ]:
# Now let's plot it on a log-log scale
ax=pop.plot(x='ind',y='pop',loglog=True)
ax.set_xlabel('Rank')
ax.set_ylabel('Population');


In [ ]:
# And how about just the ones where the rank is <= 100?
# This is for countries > ~675,000 people
pop_sub = pop[pop['ind'] < 100]
ax=pop_sub.plot(x='ind',y='pop',loglog=True)
ax.set_xlabel('Rank')
ax.set_ylabel('Population')

Zipf's law as originally formulated has the form:   p = r<sup>-1</sup>

On the last log-log plot for counties >= ~675,000 we see a straight line, suggesting a power law relationship.

Let's do a fit of the form p = B r<sup>A</sup> and see what we get for the fitting parameters A and B.

Two ways:
* Linear least squares after taking the log (base 10) of the equation:  log10(p) = A log10(r) + B<sub>0</sub>
* Non-linear least squares to p = r<sup>A</sup>

In [ ]:
# Linear first.  Use a convenient function from scipy:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
result = sp.stats.linregress(np.log10(pop_sub['ind']), np.log10(pop_sub['pop']))
# What's in the result?  Check the docs
lin_A = result.slope
lin_B = 10**result.intercept
print('A = %1.4f  B = %3.4f'  % (lin_A, lin_B))        


In [ ]:
# Non-linear without taking the log of the equation. This uses another scipy function:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html
# This requires a Python function that computes residuals, i.e. the difference between
# the result of the equation with the guessed parameter A and the y values.
def residuals(coeffs, x, y):
    # coeffs in an array of size 1 
    # coeffs[0] --> A
    # coeffs[1] --> B
    return y - coeffs[1] * x**coeffs[0]

# An initial guess is needed for A. Let's try -1.
# To get this to work we'll need to force it to only consider values for A less than 1.
result = sp.optimize.least_squares(residuals, [-1.0,1.0], args=(pop_sub['ind'], pop_sub['pop']))
nonlin_A = result.x[0]
nonlin_B = result.x[1]
print('A = %1.4f  B = %3.4f'  % (nonlin_A, nonlin_B))        


In [ ]:
# Let's plot the results!
x0 = pop_sub['ind']
y0 = pop_sub['pop']
y1 = lin_B * x0**lin_A
y2 = nonlin_B * x0**nonlin_A
plt.loglog(x0,y0,'.')
plt.loglog(x0,y1,'-')
plt.loglog(x0,y2,'-')
plt.legend(['data','linear lsq','non-lin lsq'])
plt.xlabel('Rank')
plt.ylabel('County Population')

In [ ]:
# Now...I wonder if this holds for States too...
state_pop = df.groupby('State').sum()['pop'].copy()
state_pop.sort_values(ascending = False, inplace=True)
state_ind = np.arange(1,52) # numbers 1 to 51...including Washington, D.C.
# Fit to the 30 biggest states
state_res = sp.stats.linregress(np.log10(state_ind[0:30]), np.log10(state_pop.iloc[0:30]))
# What's in the result?  Check the docs
lin_A = state_res.slope
lin_B = 10**state_res.intercept
plt.figure()
plt.loglog(state_ind, state_pop,'.')
plt.loglog(state_ind, lin_B * state_ind**lin_A)
plt.legend(['data','linear lsq'])
print('A = %3.4f   B=%3.4f' % (lin_A, lin_B))

# Hypothesis Testing

* State Hypothesis
* Formulate an analysis plan
* Perform analysis
* Interpret results

## Checking if a variable is normally distributed
Let's look at life expectancy...

In [ ]:
fig, ax = plt.subplots()
# Uncomment this line and comment the sns line to compare plotting styles
#df.hist('Life_Expectancy',ax=ax, bins=np.arange(60, 100, 0.5))
ax.hist(df['life_exp'], bins=np.arange(60, 100, 0.5))
med_age = df['life_exp'].median()
# How high to plot the median line?  Let's query the axes for
# the ylimits:
ylim = ax.get_ylim() # ylim == (0.0, 1167.6)
ax.plot([med_age,med_age],[0,ylim[1]],'r--')
ax.set_xlabel('age')
ax.set_ylabel('Count of counties')
ax.set_title('Life Expectancy')
# Set the y axis to the range it was at before we added the median line
ax.set_ylim(ylim)
ax.set_xlim([60,100])
ax.legend(['Median age: %2.1f' % med_age])

In [ ]:
# Is this normally distributed?  Let's overlay a Gaussian curve...
# Get the histogram values without any nan values.   
clean_life_exp = df[df['life_exp'] > 0]['life_exp'] 

from scipy.optimize import curve_fit
# The curve_fit function is a general non-linear fitting tool.  It takes
# a function as before with least_squares above.
hist, bin_edges = np.histogram(clean_life_exp, density=True, bins=np.arange(60,100,0.5))
bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

# Define model function to be used to fit to the data above:
def gauss(x, A, mu, std):
    return A*np.exp(-(x-mu)**2/(2.*std**2))

# Pick some guesses for A, mu, std
p0 = [1., np.mean(clean_life_exp), 1]
coeff, var_matrix = curve_fit(gauss, bin_centers, hist, p0=p0)
# Extract the coefficients
A, mu,std=coeff 

In [ ]:
# And plot it
fig, ax = plt.subplots()
# density=True will normalize the plot of the histogram
ax.hist(df['life_exp'],  density=True, bins=np.arange(60, 100, 0.5))
med_age = df['life_exp'].median()
# How high to plot the median line?  Let's query the axes for
# the ylimits:
ylim = ax.get_ylim() # ylim == (0.0, 1167.6)
ax.plot([med_age,med_age],[0,ylim[1]],'r--')
ax.set_xlabel('age')
ax.set_ylabel('Count of counties')
ax.set_title('Life Expectancy')
# Set the y axis to the range it was at before we added the median line
ax.set_ylim(ylim)
ax.set_xlim([60,100])
ax.legend(['Median age: %2.1f' % med_age])

# Use the norm sub-sub-library from scipy to compute the PDF for the plot.
from scipy.stats import norm
hist, bin_edges = np.histogram(clean_life_exp, density=True, bins=np.arange(60, 100, 0.5))
p = norm.pdf(bin_edges, mu, std)
ax.plot(bin_edges, p, 'k', linewidth=2)
# Let's check whether this fits well more rigourously...

### State Hypothesis:
* Null-hypothesis - the sample comes from a normal distribution. 
* Alternative hypothesis - the sample does not come from a normal distribution

### Formulate an analysis plan

We will run [Shapiro-Wilks](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) test of normality.

### Perform analysis
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

Scipy has a Shapiro-Wilks test included.

In [ ]:
sp.stats.shapiro(df['life_exp'])

In [ ]:
# Looks like we still need to remove the NaN values!
clean_life_exp = df[df['life_exp'] > 0]['life_exp']
result = sp.stats.shapiro(clean_life_exp)
# What's in the result?  try: dir(result)
print('p-value: %3.5e' % result.pvalue)

### Interpret Results

In [ ]:
# Despite all the very high-level functions we're using don't forget we're still 
# writing regular Python code!  All the regular Python language features are available
# all the time, of course.  Let's have Python figure out the answer...
if result.pvalue  < 0.05:
    print('Null hypothesis is rejected, not from a normal distribution.')
else:
    print('Null hypothesis is accepted, samples are from a normal distribution.')


## Comparing two samples

Let’s compare life expectancy in New England and East South Ctr Region. Let’s first summarize our data and find median and mean values.

In [ ]:
# Group, extract just the life_exp column to avoid doing more calculations than necessary,
# do the calculations.
df.groupby('Region')[['life_exp']].agg(['mean','median'])

In [ ]:
# Let’s visually compare two distributions of this variable for both regions
# Make a view onto just the two regions of interest
reg_df = df[df['Region'].isin(['East South Ctr','New. Eng.'])][['Region','life_exp']]
reg_df 

In [ ]:
# Now to put two histograms onto on axes...
bins = np.arange(60,100,0.5)
# Two sets of data
esc = reg_df[reg_df['Region']=='East South Ctr']['life_exp']
ne = reg_df[reg_df['Region'] == 'New. Eng.']['life_exp']

# Let's check to see if distributions for each region
# are normally distributed...

# Put repetitive things into functions.
def print_is_norm(var, name):
    result = sp.stats.shapiro(ne)
    if result.pvalue  < 0.05:
        print('%s is not from a normal distribution.' % name)
    else:
        print('%s samples is from a normal distribution.' % name)
    
print_is_norm(esc, 'East South Ctr')
print_is_norm(ne, 'New. Eng.')


In [ ]:
# Note a new way to do a vertical line...
plt.hist(esc, bins, alpha=0.5, label='East South Ctr')
# **** Why was this variable created?
esc_avg = esc.mean()
plt.axvline(esc_avg, color='k', linestyle='dashed', linewidth=1)
plt.hist(ne, bins, alpha=0.5, label='New. Eng.')
ne_avg = ne.mean()
plt.axvline(ne_avg, color='k', linestyle='dashed', linewidth=1)

plt.legend(loc='upper right')
# Add labels to the median lines
min_ylim, max_ylim = plt.ylim()
# Positioning this text is a little laborious. text() takes 3 args:
# x-position / y position to start the text, and a string
plt.text(esc_avg*0.86, max_ylim*0.9, 'Mean: %1.2f' % (esc_avg,))
plt.text(ne_avg*1.005, max_ylim*0.5, 'Mean: %1.2f' % (ne_avg,))

### State Hypothesis:
* Null-hypothesis -  difference in means is equal to 0. 
* Alternative hypothesis - true difference in means is not equal to 0.

In other words, is the difference in mean value between these two regions statistically significant?

### Formulate an analysis plan

We will run [Welch's Two Sample t-test](https://en.wikipedia.org/wiki/Welch%27s_t-test).  This applies when two sets of samples are both from normal distributions.


### Perform analysis
Scipy routine: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html


In [ ]:
sp.stats.ttest_ind(esc, ne, equal_var=False)

In [ ]:
# Hmmm...nan values are still causing trouble.  This particular
# function has an option to handle them
sp.stats.ttest_ind(esc, ne, equal_var=False, nan_policy='omit')


### Interpret results

From the output, the p-value < 0.05 implying that the true difference in means is not equal to zero and we can reject the null hypothesis. We then conclude that the difference in mean values in the two regions is significant.

### Exercise for Later

* Check if the variable house_income normally distributed
* Perform Welch t.test to compare house_income between New England Region and East South Ctr


## Correlation and Covariance

Let's go back and consider the entire dataset. Pandas can compute a correlation matrix:

In [ ]:
# The default is the Pearson method
corr = df.corr()
corr

In [ ]:
# How about in visual form?
plt.matshow(corr)
plt.colorbar()
# Our dataset mixes health info with demographics so this is slightly funky looking

In [ ]:
# Seaborn version
sns.heatmap(corr)

In [ ]:
# Seaborn version again.  This time, overwrite the diagonal 
# 1's in the correlation matrix with NaN values so that they don't 
# get displayed.

# corr.values will access the underlying Numpy storage for the 
# dataframe.  As this is a dataframe entirely of float64 types it's
# stored as a single 2D numpy array:
print(type(corr.values))
print(corr.shape)

In [ ]:
# Use a numpy function to overwrite the diagonals
#  https://numpy.org/doc/stable/reference/generated/numpy.fill_diagonal.html
# This modification is in-place.  Functions like this are likely implemented
# in compiled C code and are much faster than doing this using Python for loops.
np.fill_diagonal(corr.values, np.nan)

In [ ]:
# replot
sns.heatmap(corr)
plt.matshow(corr)
plt.colorbar()


In [ ]:
# The % Fair Health vs % Smokers has a high correlation - 0.728878
# Just to show it off, plot with seaborn with a straight line fit
# https://seaborn.pydata.org/generated/seaborn.regplot.html
sns.regplot(x='% Smokers',y='% Fair or Poor Health',data=df, fit_reg=True, line_kws={"color": "red"})

In [ ]:
# Calculate the correlation between these columns
corr = df['% Smokers'].corr(df['% Fair or Poor Health'])
corr

In [ ]:
# Now the covariance
covar = df['% Smokers'].cov(df['% Fair or Poor Health'])
covar

In [ ]:
#  Pearson correlation coefficient and p-value 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

r, p_value = sp.stats.pearsonr(df['% Smokers'],df['% Fair or Poor Health'])
print("Pearson's correlation coefficient: %s    P-value: %s" % (r,p_value))

## Pivot Table

We'll add a column to our dataframe called "wealth" which categorizes household income. Then let's make a new dataframe using a spreadsheet-style pivot table.

In [ ]:
# Categorize household income: 
#    < 50,000  : low
#    < 80,000  : high
#    in between: middle

# Loop over the dataframe rows, build a list of strings, add it
# as a new column
# Let's look at some different ways to do this...

def method_iterrows(df):
    wealth = []  # Empty Python list
    for row in df.iterrows():
        # This returns each row as:
        # (row_number, Series)
        house_inc = row[1]['house_income']
        cat = 'middle'
        if house_inc < 50000:
            cat = 'low' 
        elif house_inc > 80000:
            cat = 'high' 
        wealth.append(cat)
    return wealth

def method_itertuples(df):
    wealth = []  # Empty Python list
    for row in df.itertuples():
        # This returns each row as a "namedtuple"
        # https://pymotw.com/2/collections/namedtuple.html
        house_inc = row.house_income
        cat = 'middle'
        if house_inc < 50000:
            cat = 'low' 
        elif house_inc > 80000:
            cat = 'high' 
        wealth.append(cat)
    return wealth

def method_col_loop(df):
    wealth = []  # Empty Python list
    # Loop over the elements of a column
    for house_inc in df['house_income']:
        # house_inc is just each value in this column
        cat = 'middle'
        if house_inc < 50000:
            cat = 'low' 
        elif house_inc > 80000:
            cat = 'high' 
        wealth.append(cat)
    return wealth

In [ ]:
# %time is a special iPython command that gives an elapsed time for a function call
# You can also try %timeit which runs the command multiple times and gives an average elapsed time
%time method_iterrows(df)
%time method_itertuples(df)
%time method_col_loop(df)


In [ ]:
# Add the new column
df['wealth'] = method_col_loop(df)
df

In [ ]:
# Group by region and count the values of wealth in each region.
# The function here is pivot_table
# hmmm...couldn't get this to work without a new column 
df['wealth_'] = df['wealth']
wealth_df = df.pivot_table(index='Region',values='wealth',columns='wealth_',aggfunc='count')
wealth_df

In [ ]:
wealth_df.columns